## Medsam Implementation

### Disclaimer: 

This notebook includes text generated by a large language model (LLM). While I have reviewed and edited the content to the best of my ability, it is important to acknowledge the potential for inaccuracies. I encourage you to cross-check any critical information with independent sources.

### Download Data

In [ ]:
# prompt: write me a wget code to get file from my googld drive

!wget --no-check-certificate 'https://drive.usercontent.google.com/download?id=1cfORNOh_p1KHbhw8U0zhGkjSC2IrGaTB&export=download&confirm=t&uuid=c5760839-02fd-436b-9e38-9635cd7d8aab&at=APZUnTVLTsvNxsW4SInOAOYrK9ZA%3A1715907782351' -O images.zip


### Data Preprocessing

In [ ]:
%cd /kaggle/input/medsam-github-repository

In [ ]:
#✅ Step 1: Add MedSAM to Python path
import sys
sys.path.append("/kaggle/input/medsam-github-repository/MedSAM-main")
#sys.path.append("/kaggle/input/medsam-github-repository/MedSAM-main/MedSAM_Inference.py")
sys.path.append("/kaggle/input/medsam-model-checkpoint/medsam_vit_b.pth")

In [ ]:
IMAGE_PATH = '/kaggle/input/dataset/kaggle_3m'


In [ ]:
## source: https://www.kaggle.com/code/saeedghamshadzai/image-segmentation-brain-tumor-u-net-cnn#Inferece
import os
paths = []

for dirname in os.listdir(IMAGE_PATH):
    if os.path.isdir(os.path.join(IMAGE_PATH, dirname)):

        for filename in os.listdir(os.path.join(IMAGE_PATH, dirname)):
            # Only the files with ',tif' format should be added to the 'paths' list
            if filename.endswith('.tif'):
                paths.append(IMAGE_PATH+'/'+dirname+'/'+filename)

len(paths), paths[:20:5]

MEDSAM IMPLEMENTATION

In [ ]:
# This is from the actual MedSam Inference python file
import torch
import numpy as np
import cv2
from torchvision import transforms
from segment_anything import sam_model_registry
from segment_anything.utisl.transforms import ResizeLongestSide



def medsam_inference(medsam_model, img_embed, box_1024, H, W):
    box_torch = torch.as_tensor(box_1024, dtype=torch.float, device=img_embed.device)
    if len(box_torch.shape) == 2:
        box_torch = box_torch[:, None, :]  # (B, 1, 4)

    sparse_embeddings, dense_embeddings = medsam_model.prompt_encoder(
        points=None,
        boxes=box_torch,
        masks=None,
    )
    low_res_logits, _ = medsam_model.mask_decoder(
        image_embeddings=img_embed,  # (B, 256, 64, 64)
        image_pe=medsam_model.prompt_encoder.get_dense_pe(),  # (1, 256, 64, 64)
        sparse_prompt_embeddings=sparse_embeddings,  # (B, 2, 256)
        dense_prompt_embeddings=dense_embeddings,  # (B, 256, 64, 64)
        multimask_output=False,
    )

    low_res_pred = torch.sigmoid(low_res_logits)  # (1, 1, 256, 256)

    low_res_pred = F.interpolate(
        low_res_pred,
        size=(H, W),
        mode="bilinear",
        align_corners=False,
   )  # (1, 1, gt.shape)
    low_res_pred = low_res_pred.squeeze().cpu().numpy()  # (256, 256)
    medsam_seg = (low_res_pred > 0.5).astype(np.uint8)
    return medsam_seg

In [ ]:
# This is a generated function for a single image


# --- Function to run MedSAM on a single image + bounding box ---
def medsam_infer_single_image(image: np.ndarray, box: list, model, device):
    """
    Args:
        image (np.ndarray): Input image as HxW or HxWx3 (grayscale or RGB).
        box (list): [x0, y0, x1, y1] bounding box in pixel coords.
        model: Loaded MedSAM model.
        device: torch.device("cuda" or "cpu")
    
    Returns:
        pred_mask (np.ndarray): Output binary mask (0 or 1), shape (H, W)
    """
    
    original_size = image.shape[:2]

    # Make grayscale into RGB if needed
    if image.ndim == 2:
        image = np.stack([image] * 3, axis=-1)

    # Resize to 1024 (MedSAM input size)
    transform = ResizeLongestSide(1024)
    image_resized = transform.apply_image(image)
    image_tensor = torch.as_tensor(image_resized.transpose(2, 0, 1)).float().to(device) / 255
    image_tensor = image_tensor.unsqueeze(0)

    # Resize and normalize the box
    box_np = np.array(box)[None, :]
    box_tensor = torch.as_tensor(transform.apply_boxes(box_np, original_size), device=device)

    # Get image embedding
    with torch.no_grad():
        image_embedding = model.image_encoder(image_tensor)

        sparse_embeddings, dense_embeddings = model.prompt_encoder(
            points=None,
            boxes=box_tensor,
            masks=None,
        )

        low_res_masks, _ = model.mask_decoder(
            image_embeddings=image_embedding,
            image_pe=model.prompt_encoder.get_dense_pe(),
            sparse_prompt_embeddings=sparse_embeddings,
            dense_prompt_embeddings=dense_embeddings,
            multimask_output=False,
        )

        low_res_masks = torch.sigmoid(low_res_masks)
        low_res_masks = low_res_masks.squeeze().cpu().numpy()

        # Resize back to original size
        pred_mask = cv2.resize((low_res_masks > 0.5).astype(np.uint8), (original_size[1], original_size[0]), interpolation=cv2.INTER_NEAREST)
    
    return pred_mask


In [ ]:
## source https://www.kaggle.com/code/saeedghamshadzai/image-segmentation-brain-tumor-u-net-cnn#Inferece
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
def data_frame(data):
    # Storing only paths that don't end with 'mask.tiff'
    images = list(filter(lambda x: not x.endswith('mask.tif'), data))
    # Sorting images based on the number of each MRI.
    images.sort(key=lambda x: int(x.rsplit('_', 3)[-1][:-4]))
    # Sorting by the patient IDs (each patient has more than 1 MRIs)
    images.sort(key=lambda x: int(x.rsplit('_', 3)[-2]))

    # Storing the image IDs
    IDs = list(map(lambda x: x.rsplit('/', 3)[-1][:-4], images))

    # Storing only paths that end with 'mask.tiff'
    masks = list(filter(lambda x: x.endswith('mask.tif'), data))
    # Sorting masks based on the number of each MRI.
    masks.sort(key=lambda x: int(x.rsplit('_', 3)[-2]))
    # Sorting by the patient IDs (each patient has more than 1 MRIs)
    masks.sort(key=lambda x: int(x.rsplit('_', 3)[-3]))

    # Opens the images
    pixels = lambda x: Image.open(x)
    # Selects the largest pixel
    largest_pixel = lambda y: np.max(pixels(y))
    # Determines if the mask contains an abnormality or not (+ or -)
    # Remember that a negative image's mask is just an entirely black image.
    diagnotic_function = lambda z: 1 if largest_pixel(z) > 0 else 0
    # Storing the diagnosis corresponding to each image
    diagnoses = list(map(lambda x: diagnotic_function(x), masks))

    # Making the dataframe
    DataFrame = pd.DataFrame({'ID': IDs, 'Image': images, 'Mask': masks, 'Diagnosis': diagnoses})

    # Dividing the indexes into train, test, and validation
    train_index, test_index = train_test_split(DataFrame.index.values.tolist(), test_size=0.15, random_state=42)
    train_index, val_index = train_test_split(train_index, test_size=0.1, random_state=42)

    # Making train, test, and validation dataframes
    train_df, val_df, test_df = DataFrame.iloc[train_index], DataFrame.iloc[val_index], DataFrame.iloc[test_index]

    return train_df, val_df, test_df

# Making the dataframes
train_df, val_df, test_df = data_frame(paths)

print(len(train_df), len(val_df), len(test_df))

train_df.head()

In [ ]:
from segment_anything import sam_model_registry
import torch

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 1: Initialize model WITHOUT checkpoint
sam_model = sam_model_registry["vit_b"](checkpoint=None)

# Step 2: Load weights manually to the correct device
state_dict = torch.load("/kaggle/input/medsam-model-checkpoint/medsam_vit_b.pth", map_location=device)
sam_model.load_state_dict(state_dict)

# Step 3: Move model to device
sam_model.to(device)
sam_model.eval()

In [ ]:
##### Sample Image + Mask Loader for MedSAM

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# --- Step 1: Select an image from your test set ---
sample = test_df[test_df['Diagnosis'] == 1].iloc[0]  # pick one with tumor
img_path = sample['Image']
mask_path = sample['Mask']

# --- Step 2: Load and resize image & mask to 256x256 ---
img = Image.open(img_path).convert("L").resize((256, 256))
mask = Image.open(mask_path).convert("L").resize((256, 256))

img_np = np.array(img)
mask_np = np.array(mask)

# --- Step 3: Create bounding box from mask ---
def get_bounding_box(mask):
    """Returns [x0, y0, x1, y1] box around non-zero mask pixels"""
    y_indices, x_indices = np.where(mask > 0)
    if len(x_indices) == 0 or len(y_indices) == 0:
        return [0, 0, 0, 0]  # empty mask
    return [int(x_indices.min()), int(y_indices.min()),
            int(x_indices.max()), int(y_indices.max())]

bbox = get_bounding_box(mask_np)

# --- Step 4: Show image, mask, and bounding box for sanity check ---
plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.title("Image")
plt.imshow(img_np, cmap='gray')

plt.subplot(1, 3, 2)
plt.title("Mask")
plt.imshow(mask_np, cmap='gray')

plt.subplot(1, 3, 3)
plt.title("Bounding Box")
plt.imshow(img_np, cmap='gray')
x0, y0, x1, y1 = bbox
plt.gca().add_patch(plt.Rectangle((x0, y0), x1-x0, y1-y0, edgecolor='red', facecolor='none', lw=2))

plt.tight_layout()
plt.show()



In [ ]:
from segment_anything.utils.transforms import ResizeLongestSide

pred_mask = medsam_infer_single_image(img_np, bbox, sam_model, device)

In [ ]:
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(img_np, cmap='gray')

plt.subplot(1, 2, 2)
plt.title("MedSAM Predicted Mask")
plt.imshow(pred_mask, cmap='gray')

plt.show()

In [ ]:
import torch

def iou_score(pred, target, smooth=1e-6):
    pred = torch.tensor(pred).float()
    target = torch.tensor(target).float()
    intersection = torch.sum(pred * target)
    union = torch.sum(pred) + torch.sum(target) - intersection
    return ((intersection + smooth) / (union + smooth)).item()

def dice_score(pred, target, smooth=1e-6):
    pred = torch.tensor(pred).float()
    target = torch.tensor(target).float()
    intersection = torch.sum(pred * target)
    return ((2 * intersection + smooth) / (torch.sum(pred) + torch.sum(target) + smooth)).item()

# Binarize ground truth mask (in case it's not 0/1)
gt_mask_bin = (mask_np > 0).astype(np.uint8)

# Calculate metrics
iou = iou_score(pred_mask, gt_mask_bin)
dice = dice_score(pred_mask, gt_mask_bin)

print(f"MedSAM IoU Score: {iou:.4f}")
print(f"MedSAM Dice Score: {dice:.4f}")

In [ ]:
ious = []
dices = []

# You can adjust how many images to test with
num_samples = 10
samples = test_df[test_df['Diagnosis'] == 1].iloc[:num_samples]

for idx, row in samples.iterrows():
    img_path = row['Image']
    mask_path = row['Mask']
    
    # Load and resize image & mask
    img = Image.open(img_path).convert("L").resize((256, 256))
    mask = Image.open(mask_path).convert("L").resize((256, 256))
    
    img_np = np.array(img)
    mask_np = np.array(mask)
    gt_mask_bin = (mask_np > 0).astype(np.uint8)
    
    # Get bounding box from ground truth mask
    def get_bounding_box(mask):
        y_indices, x_indices = np.where(mask > 0)
        if len(x_indices) == 0 or len(y_indices) == 0:
            return [0, 0, 0, 0]  # no tumor
        return [int(x_indices.min()), int(y_indices.min()), int(x_indices.max()), int(y_indices.max())]
    
    box = get_bounding_box(gt_mask_bin)
    
    if box == [0, 0, 0, 0]:
        continue  # skip empty masks
    
    # Predict with MedSAM
    pred_mask = medsam_infer_single_image(img_np, box, sam_model, device)
    
    # Evaluation
    iou = iou_score(pred_mask, gt_mask_bin)
    dice = dice_score(pred_mask, gt_mask_bin)
    
    ious.append(iou)
    dices.append(dice)

# Print results
mean_iou = sum(ious) / len(ious)
mean_dice = sum(dices) / len(dices)

print(f"\n✅ MedSAM Evaluation on {len(ious)} images")
print(f"Average IoU: {mean_iou:.4f}")
print(f"Average Dice: {mean_dice:.4f}")


### Visualizing

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
index = np.arange(train_df.shape[0])
rng = np.random.default_rng(37)
rng.shuffle(index)
for idx in range(10):
    image_path = train_df.iloc[index[idx]]['Image']
    mask_path = train_df.iloc[index[idx]]['Mask']
    image = Image.open(image_path).convert('RGB')
    mask = Image.open(mask_path).convert('L')
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(image)
    ax[1].imshow(mask)

### Training the Model

U-Net Model

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

In [ ]:
# --- Use same naming and flow as in your notebook ---
class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'Image']
        mask_path = self.df.loc[idx, 'Mask']

        img = Image.open(img_path).convert('L')
        mask = Image.open(mask_path).convert('L')

        if self.transform:
            img = self.transform(img)
            mask = self.transform(mask)

        mask = (mask > 0).float()

        return img, mask

In [ ]:
# --- U-Net Architecture (unchanged, just renamed class to match style) ---
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        def CBR(in_ch, out_ch):
            return nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
                nn.ReLU(inplace=True)
            )

        self.enc1 = CBR(1, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = CBR(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = CBR(128, 256)
        self.pool3 = nn.MaxPool2d(2)

        self.bottleneck = CBR(256, 512)

        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = CBR(512, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = CBR(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = CBR(128, 64)

        self.final = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool1(e1))
        e3 = self.enc3(self.pool2(e2))

        b = self.bottleneck(self.pool3(e3))

        d3 = self.dec3(torch.cat([self.up3(b), e3], dim=1))
        d2 = self.dec2(torch.cat([self.up2(d3), e2], dim=1))
        d1 = self.dec1(torch.cat([self.up1(d2), e1], dim=1))

        return torch.sigmoid(self.final(d1))

In [ ]:
# --- Transforms to match 256x256 grayscale ---
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

In [ ]:
# --- Create datasets using your data_frame() function outputs ---
train_dataset = CustomDataset(train_df, transform=transform)
val_dataset = CustomDataset(val_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [ ]:
# --- Model setup and training ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()

In [ ]:
# --- Training loop (minimal) ---
def train_loop(model, loader):
    model.train()
    total_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_hat = model(x)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [ ]:
# --- Run training ---
for epoch in range(10):
    epoch_loss = train_loop(model, train_loader)
    print(f"Epoch {epoch+1}/10 - Loss: {epoch_loss:.4f}")

Evaluating the performance

This section evaluates our model's segmentation performance using two common metrics: Intersection over Union (IoU) and Dice coefficient (Dice). Both metrics quantify the overlap between predicted and ground truth segmentation masks.

IoU focuses on the ratio of correctly classified pixels, while Dice emphasizes the balance between true positives and both false positives and negatives. We'll calculate these scores to assess the model's segmentation accuracy.

In [ ]:
import torch

def iou_score(y_true, y_pred, smooth=1e-6):
    intersection = torch.sum(y_true * y_pred)
    union = torch.sum(y_true) + torch.sum(y_pred) - intersection
    iou = (intersection + smooth) / (union + smooth)
    
    return iou

def dice_score(y_true, y_pred, smooth=1e-6):
    intersection = torch.sum(y_true * y_pred)
    dice = (2. * intersection + smooth) / (torch.sum(y_true) + torch.sum(y_pred) + smooth)
    #print(dice)
    return dice


In [ ]:
def eval_model(model, test_loader, device):
    model.eval()
    ious = []
    dices = []

    with torch.no_grad():
        for images, masks in test_loader:
            images = images.float().to(device)
            masks = masks.float().to(device)
            outputs = model(images)
            
            outputs[outputs < .5] = 0
            outputs[outputs > .5] = 1

            iou_score_val = iou_score(outputs, masks)
            dice_score_val = dice_score(outputs, masks)

            ious.append(iou_score_val)
            dices.append(dice_score_val)

    mean_iou = sum(ious) / len(ious)
    mean_dice = sum(dices) / len(dices)


    return mean_iou, mean_dice



In [ ]:
#test_dataset = CustomDataset(test_df[test_df['Diagnosis'] == 1]['Image'].values.tolist(), test_df[test_df['Diagnosis'] == 1]['Mask'].values.tolist(), transform=transform)
#test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
#eval_model(model, test_dataloader, device)

test_df_pos = test_df[test_df['Diagnosis'] == 1]
test_dataset = CustomDataset(test_df_pos, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
eval_model(model, test_dataloader, device)

In [ ]:
torch.save(model.state_dict(), 'baseline_ckpt.pth')

In [ ]:
model.load_state_dict(torch.load('baseline_ckpt.pth'))